In [15]:
import sqlite3
with sqlite3.connect('jobs.db') as conn:
    c = conn.cursor()
    c.close()

In [ ]:

from db import DB_NAME
import sqlite3, os
from scrapers.db import initialise_db
#initialise database if not exists
if not os.path.exists(DB_NAME):
    initialise_db(DB_NAME) 



In [105]:
job_listings[10].find('a').get('href')

'/careers/jobs/R0014527/'

In [98]:
''.join([i.strip() for i in job_listings[10].select('.c-job__col')[1].children if isinstance(i,str) ])

'Melbourne, Australia'

In [12]:
job_listings[10]

<li class="l-job-listing__item" data-search-result="">
<a class="c-job" data-utag="current-jobs-list" href="/careers/jobs/R0014527/">
<div class="c-job__role">
<div class="c-job__title">
<span class="sr-only">Position: </span>Engineering Manager                                                        </div>
<div class="c-job__terms">
<span class="sr-only">Hours: </span>Full time                                <span aria-hidden="true">-</span>
<span class="sr-only">Duration: </span>Permanent                            </div>
</div>
<div class="c-job__metadata">
<div class="c-job__col c-job__col--small">
<span class="sr-only">Team: </span>
<i class="c-job__icon c-icon c-icon--large icon-multi-user" data-grunticon-embed=""></i>
                                    Technology                                </div>
<div class="c-job__col">
<span class="sr-only">Location: </span>
<i class="c-job__icon c-icon c-icon--large icon-mappin" data-grunticon-embed=""></i>
                               

In [126]:
job_listings[0].select('a')

[]

In [11]:
type(soup)

bs4.BeautifulSoup

In [7]:
from bs4 import BeautifulSoup
with open(r"./sample_pages/REA Group response text old.htm", "r", encoding="utf-8") as f:
    html_content = f.read()
soup = BeautifulSoup(html_content, 'html.parser')
jobs = []

# Find all job listings
job_listings = soup.select('.l-job-listing__item')
for listing in job_listings:
    jobs.append(listing)

In [110]:
jobs

[]

In [2]:
from scrapers.culture_amp_scraper import CultureAmpScraper


scraper = CultureAmpScraper("CultureAmp", "https://www.cultureamp.com/company/careers#open-roles")
jobs = scraper.fetch_jobs()

Skipping non-job listing
Skipping non-job listing


In [6]:
new_jobs = scraper.get_new_jobs(jobs, DB_NAME)


In [9]:
save_jobs = scraper.save_jobs(new_jobs, DB_NAME)

In [3]:
with sqlite3.connect('jobs.db') as conn:

    c = conn.cursor()
    c.execute("SELECT title, location, link FROM job where company = ?", (1,))
    result = c.fetchall() #list of tuples


In [4]:
result

[]

In [5]:
# Append job details to jobs list dataframe
import pandas as pd
jobs_df = pd.DataFrame(columns=["Title", "Location", "Link"])
for job in jobs:
    title, location = job.select('p')
    link = job.get('href')
    jobs_df = pd.concat([jobs_df, pd.DataFrame({"Title": [title.text.strip()], "Location": [location.text.strip()], "Link": [link]})], ignore_index=True)

#save into db
scraper.save_jobs(jobs_df, DB_NAME)